In [2]:
import numpy as np

class RNN:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # 初始化权重和偏差
        self.W_h = np.random.randn(hidden_dim, hidden_dim)
        self.W_x = np.random.randn(input_dim, hidden_dim)
        self.W_y = np.random.randn(hidden_dim, output_dim)
        self.b_h = np.zeros((1, hidden_dim))
        self.b_y = np.zeros((1, output_dim))
        
    def forward(self, x):
        # 计算隐藏状态
        self.h = np.zeros((1, self.hidden_dim))
        self.hidden_states = []
        for t in range(len(x)):
            self.h = np.tanh(np.dot(x[t], self.W_x) + np.dot(self.h, self.W_h) + self.b_h)
            self.hidden_states.append(self.h)
        
        # 计算输出
        self.y = np.dot(self.h, self.W_y) + self.b_y
        return self.y
    
    def backward(self, x, y_true, learning_rate):
        # 计算输出误差
        d_y = y_true - self.y
        
        # 计算隐藏状态误差
        d_h = np.zeros((1, self.hidden_dim))
        d_W_y = np.zeros_like(self.W_y)
        d_W_h = np.zeros_like(self.W_h)
        d_W_x = np.zeros_like(self.W_x)
        d_b_y = np.zeros_like(self.b_y)
        d_b_h = np.zeros_like(self.b_h)
        for t in reversed(range(len(x))):
            d_W_y += np.outer(self.hidden_states[t], d_y)
            d_b_y += d_y
            d_h = np.dot(d_y, self.W_y.T) + np.dot(d_h, self.W_h.T)
            d_h_raw = (1 - self.hidden_states[t] ** 2) * d_h
            d_W_x += np.outer(x[t], d_h_raw)
            d_W_h += np.outer(self.hidden_states[t-1], d_h_raw)
            d_b_h += d_h_raw
            
        # 更新权重和偏差
        self.W_h -= learning_rate * d_W_h
        self.W_x -= learning_rate * d_W_x
        self.W_y -= learning_rate * d_W_y
        self.b_h -= learning_rate * d_b_h
        self.b_y -= learning_rate * d_b_y
